<a href="https://colab.research.google.com/github/ebob9/meraki_prisma_access/blob/main/Meraki_%2B_Prisma_Access_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Meraki Integration Basic Information {display-mode: "form"}
#!pip install panapi
#!pip install tqdm

from tqdm.notebook import tqdm
tqdm_tasks = [
    "Install Meraki SDK",
    "Install Prisma SASE SDK",
    "import libraries",
]


pbar = tqdm(total=len(tqdm_tasks), leave=False)

pbar.set_description(tqdm_tasks[pbar.n])
null = !pip install meraki > /dev/null
pbar.update(1)

pbar.set_description(tqdm_tasks[pbar.n])
null = !pip install git+https://github.com/PaloAltoNetworks/prisma-sase-sdk-python.git > /dev/null
pbar.update(1)

# pbar.set_description(tqdm_tasks[pbar.n])
# null = !pip install pandas > /dev/null
# pbar.update(1)

#print(f"tqdm - {pbar.n}")


pbar.set_description(tqdm_tasks[pbar.n])
#import pandas as pd
#import panapi
import prisma_sase
from prisma_sase import jd, jd_detailed, jdout, jdout_detailed

import meraki

from google.colab import files
from google.colab import data_table
import io
import requests
import json
import os
import ipywidgets as widgets
from copy import deepcopy
import time
import binascii
import re
email_regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')
from IPython.display import clear_output 

pbar.update(1)

pbar.close()

#@markdown ##Meraki API Authentication Information
meraki_api_key = '' #@param {type: "string"}

#@markdown ##Prisma Access API Authentication Information
pa_service_account = ''  #@param {type: "string"}
pa_secret = '' #@param {type: "string"}
pa_tsg_id = "" #@param {type: "string"}
pa_tsg_scope = "tsg_id:" + str(pa_tsg_id)
token_url = "https://auth.apps.paloaltonetworks.com/am/oauth2/access_token"

#@markdown ##General configuration items
PSK = "IamAnonRandomPSKpleaseDontUseMePickArealPSK" #@param {type:"string"}
EMAIL_TEMPLATE = "test@test.com" #@param {type:"string"}
PA_IKE_CRYPTO_PROFILE = 'CiscoISR-IKE-Crypto-Default'
PA_IPSEC_CRYPTO_PROFILE = 'CiscoISR-IPSec-Crypto-Default'

if re.fullmatch(email_regex, EMAIL_TEMPLATE):
    pass
else:
    raise ValueError("Invalid Email")

#
# Prisma SASE SDK Instantiation
#
sdk = prisma_sase.API(ssl_verify=False)
sdk.interactive.login_secret(client_id=pa_service_account, client_secret=pa_secret,
                                 tsg_id=pa_tsg_id)

#
# Meraki SDK Instantiation
#

dashboard = meraki.DashboardAPI(api_key=meraki_api_key,suppress_logging=True)

print("Done!")


  0%|          | 0/3 [00:00<?, ?it/s]

Done!


In [2]:
#@title Meraki Integration
# Globals
INTEGRATION_VERSION = 0.91
status_bar = tqdm(total=10, leave=False)
status_bar.set_description("Initalizing")
wgt_labels = []
wgt_regions = []
wgt_spns = []
wgt_force_commit = []
wgt_meraki_status = []
wgt_pa_status = []
wgt_tunnel_status = []
current_config = {}
fabricmaps = []
DEFAULT_TUNNEL_CIDR = "169.254.0.0/16"
DISABLE_STRING = "None"
MERAKI_PREAMBLE = "PRISMA"
MERAKI_SEPERATOR = '-'
PA_PREAMBLE = "MERAKI"
PA_SEPERATOR = "_"
# Fabricmap uses RN description for bindings.
FABRICMAP_ENUM = 1

# set of used enums.
USED_ENUMS = set()

# Cached global items for API calls.
meraki_orgs = []
meraki_org_id = None
meraki_networks = {}
# sort by name
meraki_network_names = []
meraki_thirdpartyvpnpeers = {}
meraki_peers = {}
pa_allocatable_regions = {}
display2region = {}
pa_remote_networks = []
pa_ike_crypto_profiles = []
pa_ipsec_crypto_profiles = []
pa_ike_gateways = []
pa_ipsec_tunnels = []
pa_available_regions = []

# previous functions

# functions

def print_red(text):
  print("\x1b[31m" + str(text) + "\x1b[0m")


def space_to_underscore(the_string):
    # Convert merki to PA strings.
    # no spaces allowed in PA strings.
    return the_string.replace(' ', '_')

def enum_in_email(email, enum):
    """
    Put enum value inside an email address (test@test.com, 4 = test4@test.com)
    :param email: email to edit
    :param enum: integer to add
    :return: email with embedded enum integer
    """
    # split email
    email_parts = email.split('@')
    # add number to email
    email_parts[0] += '.'
    email_parts[0] += str(enum)

    return '@'.join(email_parts)


def enum_from_email(enum_email):
    """
    Get enum value from inside an email address (test@test.com, 4 = test.4@test.com)
    :param enum_email: email to extract enum.
    :return: tuple - (integer from email, email with integer removed)
    """
    # split email
    enum_email_parts = enum_email.split('@')
    # get enum from email
    enum_parts = enum_email_parts[0].split('.')
    enum = enum_parts[-1]
    # now that have enum, reconstruct email
    address = "".join(enum_parts[:-1])
    domain = enum_email[1]

    try:
        return int(enum), address + '@' + domain
    except ValueError:
        return None, None

def missing_sequence(int_list):
    """
    Take a list of integers, find missing in the sequence.
    :param int_list:
    :return: list of missing integers, list of all integers
    """
    sorted_int_list = sorted(int_list)
    first = sorted_int_list[0]
    last = sorted_int_list[-1]
    return sorted(set(range(first, last + 1)).difference(sorted_int_list)), sorted(range(first, last + 1))
  

def get_pa_remote_networks(authenticated_sase_sdk):
    rn_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/remote-networks?folder=Remote%20Networks"
    resp = authenticated_sase_sdk.rest_call(url=rn_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content.get("data", [])
    else:
        return []

def post_pa_remote_networks(authenticated_sase_sdk, data):
    rn_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/remote-networks?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=rn_url, method="POST", data=data)

def put_pa_remote_networks(authenticated_sase_sdk, id, data):
    rn_url = f"https://api.sase.paloaltonetworks.com/sse/config/v1/remote-networks/{id}?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=rn_url, method="PUT", data=data)

def delete_pa_remote_networks(authenticated_sase_sdk, id):
    rn_url = f"https://api.sase.paloaltonetworks.com/sse/config/v1/remote-networks/{id}?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=rn_url, method="DELETE")

def get_pa_ike_crypto_profiles(authenticated_sase_sdk):
    ikecryptoprofile_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ike-crypto-profiles?folder=Remote%20Networks"
    resp = authenticated_sase_sdk.rest_call(url=ikecryptoprofile_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content.get("data", [])
    else:
        return []

def get_pa_ipsec_crypto_profiles(authenticated_sase_sdk):
    ipseccryptoprofile_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ipsec-crypto-profiles?folder=Remote%20Networks"
    resp = authenticated_sase_sdk.rest_call(url=ipseccryptoprofile_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content.get("data", [])
    else:
        return []

def get_pa_ike_gateways(authenticated_sase_sdk):
    ikegateway_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ike-gateways?folder=Remote%20Networks"
    resp = authenticated_sase_sdk.rest_call(url=ikegateway_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content.get("data", [])
    else:
        return []

def post_pa_ike_gateways(authenticated_sase_sdk, data):
    ikegateway_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ike-gateways?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=ikegateway_url, method="POST", data=data)

def put_pa_ike_gateways(authenticated_sase_sdk, id, data):
    ikegateway_url = f"https://api.sase.paloaltonetworks.com/sse/config/v1/ike-gateways/{id}?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=ikegateway_url, method="PUT", data=data)

def delete_pa_ike_gateways(authenticated_sase_sdk, id):
    ikegateway_url = f"https://api.sase.paloaltonetworks.com/sse/config/v1/ike-gateways/{id}?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=ikegateway_url, method="DELETE")

def get_pa_ipsec_tunnels(authenticated_sase_sdk):
    ipsectunnel_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ipsec-tunnels?folder=Remote%20Networks"
    resp = authenticated_sase_sdk.rest_call(url=ipsectunnel_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content.get("data", [])
    else:
        return []

def post_pa_ipsec_tunnels(authenticated_sase_sdk, data):
    ipsectunnel_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ipsec-tunnels?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=ipsectunnel_url, method="POST", data=data)



def put_pa_ipsec_tunnels(authenticated_sase_sdk, id, data):
    ipsectunnel_url = f"https://api.sase.paloaltonetworks.com/sse/config/v1/ipsec-tunnels/{id}?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=ipsectunnel_url, method="PUT", data=data)

def delete_pa_ipsec_tunnels(authenticated_sase_sdk, id):
    ipsectunnel_url = f"https://api.sase.paloaltonetworks.com/sse/config/v1/ike-gateways/{id}?folder=Remote%20Networks"
    return authenticated_sase_sdk.rest_call(url=ipsectunnel_url, method="DELETE")

def get_pa_shared_infra(authenticated_sase_sdk):
    sharedinfra_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/shared-infrastructure-settings?folder=Remote%20Networks"
    resp = sdk.rest_call(url=sharedinfra_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content
    else:
        return {}

def get_pa_locations(authenticated_sase_sdk):
    locations_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/locations"
    return sdk.rest_call(url=locations_url, method="GET")


def get_pa_bandwidth_allocations(authenticated_sase_sdk):
    bandwith_allocations_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/bandwidth-allocations"
    resp = authenticated_sase_sdk.rest_call(url=bandwith_allocations_url, method="GET")
    if resp.cgx_status:
        return resp.cgx_content.get("data", [])
    else:
        return []

def push_candidate_config(authenticated_sase_sdk):
    ipsectunnel_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/config-versions/candidate:push"
    return authenticated_sase_sdk.rest_call(url=ipsectunnel_url, method="POST", data={"description": f"PRISMA AUTO MERAKI commit, app version v{INTEGRATION_VERSION}","folders": ["Remote Networks"]})


#     {
#         "name": "us-south",
#         "allocated_bandwidth": 2000,
#         "spn_name_list": [
#             "us-south-franklin",
#             "us-south-apple",
#             "us-south-cottonwood",
#             "us-south-clover"
#         ]
#     }
# ]
# [
#     {
#         "value": "canada-central",
#         "display": "Canada Central",
#         "continent": "North America",
#         "latitude": 43.6487,
#         "longitude": -79.38545,
#         "region": "northamerica-northeast2",
#         "aggregate_region": "canada-central-toronto"
#     },


def get_pa_allocatable_regions(authenticated_sase_sdk):

    pa_allocatable_regions = {}
    display2region = {}
    # get locations
    locations_resp = get_pa_locations(authenticated_sase_sdk)
    if locations_resp.cgx_status:
        pa_locations = locations_resp.cgx_content
    else:
        # Failed.
        return pa_allocatable_regions, display2region
    
    pa_allocations = get_pa_bandwidth_allocations(authenticated_sase_sdk)

    for allocation in pa_allocations:
        agg_region = allocation.get("name")
        spn_name_list = allocation.get("spn_name_list")
        # get matching locations
        for location in [matching_locations for matching_locations in pa_locations if matching_locations.get("aggregate_region") == agg_region]:
            pa_allocatable_regions[location.get("value")] = {
                "spn_name_list": spn_name_list,
                "display": location.get("display")
            }
            display2region[location.get("display")] = location.get("value")

    return pa_allocatable_regions, display2region

def extract_tags(dictionary, key='tags'):
    # check for existance of "tags" key
    if key in dictionary:
        # tags exist, return them.
        tags = dictionary.get(key, [])
        if tags is None:
            tags = []
    else:
        tags = []

    # return unique tags
    return list(set(tags))

def refresh_fabricmap(meraki_networks, meraki_peers, pa_allocatable_regions,
                               pa_remote_networks, pa_ike_crypto_profiles, pa_ipsec_crypto_profiles,
                               pa_ike_gateways, pa_ipsec_tunnels, dashboard):
    """
    Refresh fabricmap from Meraki and PA configs.
    """
    global USED_ENUMS
    global status_bar
    fabricmap = []

    # # # # This doesn't work as SPNs are in multiple regions. Need to get region from RN.
    # build reverse spn map
    # pa_spn_to_region = {}
    # for key, value in pa_allocatable_regions.items():
    #     # print(key, value)
    #     for spn in value.get("spn_name_list"):
    #         pa_spn_to_region[spn] = key

    pa_spn_all_list = []
    pa_valid_region_list = []
    for key, value in pa_allocatable_regions.items():
        pa_valid_region_list.append(key)
        pa_spn_all_list.extend(value.get("spn_name_list", []))

    # jd(pa_spn_all_list)

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Iterating Meraki Networks")

    # Get fabricmap from Meraki
    for network in meraki_networks:
        network_name = network.get('name')
        network_tags = extract_tags(network)
        network_id = network.get('id')
        network_subnets = []
        # for tag in network_tags:
        #     if tag.startswith(MERAKI_PREAMBLE + MERAKI_SEPERATOR):
        #         # Prisma integration tag, extract SPN.
        #         candidate_spn = tag.lstrip(MERAKI_PREAMBLE + MERAKI_SEPERATOR)
        #         # Check if a valid spn
        #         if candidate_spn in pa_spn_all_list:
        #             network_pa_spn = candidate_spn
        
        # check vlans
        try:
            meraki_vlans = dashboard.appliance.getNetworkApplianceVlans(network_id)
            #print(meraki_vlans)
        except meraki.APIError as e:
            print(f"Error: {e}\n. Skipping..")
            # break out of this network (skip) if VLANs not configured.
            continue
        
        for vlan in meraki_vlans:
            subnet = vlan.get('subnet')
            if subnet:
                network_subnets.append(subnet)


        fabricmap.append({
            "sdwan_name": network_name,
            "sdwan_id": network_id,
            "sdwan_tags": network_tags,
            "sdwan_subnets": network_subnets,
            "sdwan_peer_publicip": None,
            "sdwan_config_status": None,
            "pa_spn": None,
            "pa_region": None,
            "enum": None,
            "sdwan_peer_name": None,
            "pa_rn_name": None,
            "pa_rn_id": None,
            "pa_ipsec_tunnel_name": None,
            "pa_ike_gw_name": None,
            "pa_ipsec_tunnel_id": None,
            "pa_ike_gw_id": None,
            "pa_config_status": None,
            "tunnel_status": None
        })

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Iterating Meraki Peers")

    # Check name of peers - those drive the integrations.
    for peer in meraki_peers:
        peer_name = peer.get("name", "")
        if peer_name.startswith(MERAKI_PREAMBLE + MERAKI_SEPERATOR):
            # integration tunnel!
            match_network_name = peer_name.lstrip(MERAKI_PREAMBLE + MERAKI_SEPERATOR)

            for fabricmap_entry in fabricmap:
                if fabricmap_entry.get("sdwan_name") == match_network_name:
                    # This Peer matches this meraki location!

                    # Get enum from email
                    peer_local_email = peer.get("localId")
                    #print(peer_local_email)
                    if peer_local_email:
                       email_enum, clean_email = enum_from_email(peer_local_email)
                       # add enum to used enum AND fabricmap
                       USED_ENUMS.add(email_enum)
                       fabricmap_entry["enum"] = email_enum
                       fabricmap_entry["sdwan_peer_name"] = peer_name
                    fabricmap_entry["sdwan_peer_publicip"] = peer.get("publicIp", "127.0.0.1")
                    fabricmap_entry["sdwan_config_status"] = "warn" if peer.get("publicIp", "127.0.0.1") == "127.0.0.1" else "good"

                       
        # check tags
        #peer_tags = extract_tags(peer, key="networkTags")

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Iterating Prisma Access Remote Networks")

    # Start checking SASE RNs and constituents.
    for rn in pa_remote_networks:
        rn_name = rn.get("name", "")
        rn_ipsec_tunnel = rn.get("ipsec_tunnel")
        if rn_name.startswith(PA_PREAMBLE + PA_SEPERATOR):
            # integration tunnel!
            pa_match_network_name = rn_name.lstrip(PA_PREAMBLE + PA_SEPERATOR)

            for fabricmap_entry in fabricmap:
                # RN supports spaces, no need to translate name for RN.
                if fabricmap_entry.get("sdwan_name") == pa_match_network_name:
                    # This RN matches this meraki location!
                    # set config status to bad unless all is here
                    fabricmap_entry["pa_config_status"] = "bad"

                    fabricmap_entry["pa_region"] = rn.get("region") if rn.get("region") in pa_valid_region_list else None
                    fabricmap_entry["pa_spn"] = rn.get("spn_name") if rn.get("spn_name") in pa_spn_all_list else None
                    fabricmap_entry["pa_rn_name"] = rn_name
                    fabricmap_entry["pa_rn_id"] = rn.get('id')
                    fabricmap_entry["pa_ipsec_tunnel_name"] = rn_ipsec_tunnel

                    # get ipsec tunnel info
                    # print(f"FABRICMAP_ENTRY: {fabricmap_entry['pa_ipsec_tunnel_name']}")
                    if fabricmap_entry["pa_ipsec_tunnel_name"]:
                        matching_ipsec_tunnel_list = [entry for entry in pa_ipsec_tunnels 
                                                      if (entry.get("name") == fabricmap_entry["pa_ipsec_tunnel_name"])
                                                      and (entry.get("name") is not None)]
                        # jd(pa_ipsec_tunnels )
                        if matching_ipsec_tunnel_list: # if entry, grab first.
                            ipsec_tunnel = matching_ipsec_tunnel_list[0]
                            fabricmap_entry["pa_ipsec_tunnel_id"] = ipsec_tunnel.get("id")
                            ike_gw_list = ipsec_tunnel.get("auto_key", {}).get("ike_gateway", [])
                            if ike_gw_list: # needs more error handling
                                fabricmap_entry["pa_ike_gw_name"] = ike_gw_list[0].get("name")

                    # get ike gw info
                    if fabricmap_entry["pa_ike_gw_name"]:
                        matching_ike_gw_list = [entry for entry in pa_ike_gateways 
                                                if (entry.get("name") == fabricmap_entry["pa_ike_gw_name"])
                                                and (entry.get("name") is not None)]
                        if matching_ike_gw_list: # if entry, grab first.
                            ike_gw = matching_ike_gw_list[0]
                            fabricmap_entry["pa_ike_gw_id"] = ike_gw.get("id")
                            # if we got all this, config status is good.
                            fabricmap_entry["pa_config_status"] = "good"

    # put meraki tunnel status in fabricmap.
    tunnel_status_response = dashboard.appliance.getOrganizationApplianceVpnStatuses(meraki_org_id)

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Iterating Tunnel Status")

    tunnel_status_dict = {}
    for entry in tunnel_status_response:
        for tunnels in entry.get("thirdPartyVpnPeers", []):
          tunnel_status_dict[tunnels.get("name")] = tunnels.get("reachability", "unknown")

    # jd(tunnel_status_dict)
    for fabricmap_entry in fabricmap:
        peer_tunnel_status = tunnel_status_dict.get(MERAKI_PREAMBLE + MERAKI_SEPERATOR + fabricmap_entry.get("sdwan_name"))
        # print(peer_tunnel_status)
        if peer_tunnel_status == "reachable":
          fabricmap_entry["tunnel_status"] = "good"
        elif peer_tunnel_status == "unknown":
          fabricmap_entry["tunnel_status"] = "bad"
        else:
          fabricmap_entry["tunnel_status"] = None
        
        # hack for demo of status with specific branch names
        # if fabricmap_entry['sdwan_name'] in [
        #         'Branch 1',
        #         'Branch 2',
        #         'Branch 3',
        #         'Branch 4'
        #     ]:
        #     fabricmap_entry['tunnel_status'] = fabricmap_entry['pa_config_status']
        
    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Finished Network Iteration")

    return fabricmap

def remote_gw_addr_dicts(authenticated_sase_sdk):
    remoteaddr_n2ip = {}
    remoteaddr_ip2n = {}

    # get API key
    shared_infra_resp = get_pa_shared_infra(authenticated_sase_sdk)

    header_api_key = shared_infra_resp.get("api_key", "")

    # quick and dirty requestor for PA Pub IP API.
    pa_pubip_api = prisma_sase.requests.Session()
    pa_pubip_api.headers.update({
        "header-api-key": header_api_key,
        'Content-Type': 'application/json'
    })

    pa_pubip_data = {
        "serviceType": "remote_network",
        "addrType": "service_ip",
        "location": "all"
    }

    pa_pubip_response = pa_pubip_api.request(method='POST', 
                                             url="https://api.prod.datapath.prismaaccess.com/getPrismaAccessIP/v2", 
                                             data=json.dumps(pa_pubip_data))

    try:
        goodresponse = pa_pubip_response.json()
    except ValueError:
        # was not JSON
        return False, remoteaddr_n2ip, remoteaddr_ip2n, "Unable to parse response."

    # extract good response
    status = goodresponse.get('status')
    # check for a message
    message = goodresponse.get('message')

    # set boolean return_status based on success
    return_status = False
    if status:
        if status.lower() in 'success':
            return_status = True

    results = goodresponse.get('result', {})
    for result in results:
        addrlist = result.get("address_details", [])
        for addr_entry in addrlist:
            spn_ip = addr_entry.get("address")
            for rn_name in addr_entry.get("node_name", []):
                if rn_name and spn_ip:
                    # Good entry, add entry to dicts.
                    remoteaddr_ip2n[spn_ip] = rn_name
                    remoteaddr_n2ip[rn_name] = spn_ip

    # return what we got
    return return_status, remoteaddr_n2ip, remoteaddr_ip2n, message

def generate_meraki_peers_from_fabricmaps(fabricmaps):
    candidate_meraki_peers = []
    meraki_peer_name2rn_name = {}
    for fabricmap in fabricmaps:
        if fabricmap.get("pa_spn") and fabricmap.get("pa_region") and fabricmap.get('pa_rn_name') and fabricmap.get('enum'):
            # this is a network that needs a peer.
            network_peer = deepcopy(MERAKI_PEER_TEMPLATE)
            network_peer["name"] = MERAKI_PREAMBLE + MERAKI_SEPERATOR + fabricmap["sdwan_name"]
            # fill lookup dict as well.
            meraki_peer_name2rn_name[network_peer["name"]] = fabricmap.get("pa_rn_name")
            # public ip = 127.0.0.1 = needs lookup on PA after this.
            network_peer["publicIp"] = fabricmap.get("sdwan_peer_publicip") if fabricmap.get("sdwan_peer_publicip") is not None else "127.0.0.1"
            network_peer["localId"] = enum_in_email(EMAIL_TEMPLATE, fabricmap['enum'])
            network_peer["remoteId"] = "pa-headend-" + EMAIL_TEMPLATE
            network_peer['networkTags'] = [MERAKI_PREAMBLE + MERAKI_SEPERATOR + space_to_underscore(fabricmap["sdwan_name"])]
            network_peer["secret"] = PSK

            # done
            candidate_meraki_peers.append(network_peer)

    return candidate_meraki_peers, meraki_peer_name2rn_name

def update_meraki_endpoint_ips(authenticated_sase_sdk, candidate_meraki_peers, meraki_peer_name2rn_name, force=False):
    global status_bar
    
    # Start loop to fill Meraki tunnel IPs.
    loop_count = 0
    max_loop = 10
    sleep_loop = 2
    if force:
        for peer in candidate_meraki_peers:
            # set all peers to 127.0.0.1 in this temp object to force refresh of all IPs when "force" is True.
            peer['publicIp'] = "127.0.0.1"

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Updating Meraki VPN IPs")

    initial_peers_needing_remote_ip = len([peer for peer in candidate_meraki_peers if peer.get("publicIp") == "127.0.0.1"])
    # print(initial_peers_needing_remote_ip)
    previous_peers_needing_remote_ip = initial_peers_needing_remote_ip
    peers_needing_remote_ip = initial_peers_needing_remote_ip
    # print(f"{peers_needing_remote_ip} of {initial_peers_needing_remote_ip} peers need remote IP update.")
    while peers_needing_remote_ip > 0 and loop_count <= max_loop:
        # update dicts
        # print(f"loop: {loop_count}")
        rn_ip_success, rn_name2ip, ip2rn_name, message = remote_gw_addr_dicts(authenticated_sase_sdk)
        for peer in candidate_meraki_peers:
            peer_name = peer.get("name", "")
            peer_ip = peer.get('publicIp')
            # print(f"{peer_name} : {peer.get('publicIp')}")
            # print(f'{peer_name} : {meraki_peer_name2rn_name.get(peer_name, "")}')
            # print(f'{peer_name} : {rn_name2ip.get(meraki_peer_name2rn_name.get(peer_name, ""),"127.0.0.1")}')
            if peer_ip == "127.0.0.1":
                # try to update.
                peer['publicIp'] = rn_name2ip.get(meraki_peer_name2rn_name.get(peer_name, ""),"127.0.0.1")
        time.sleep(sleep_loop)

        peers_needing_remote_ip = len([peer for peer in candidate_meraki_peers if peer.get("publicIp") == "127.0.0.1"])
        if peers_needing_remote_ip < previous_peers_needing_remote_ip:
            # update tracker and run API as we got at least one.
            previous_peers_needing_remote_ip = peers_needing_remote_ip
            meraki_peer_result = dashboard.appliance.updateOrganizationApplianceVpnThirdPartyVPNPeers(meraki_org_id, candidate_meraki_peers)
            #jd(meraki_peer_result)
            # print(f"{peers_needing_remote_ip} of {initial_peers_needing_remote_ip} peers need remote IP update.")
        loop_count += 1
        if status_bar is not None:
            status_bar.set_postfix_str(f"{initial_peers_needing_remote_ip - peers_needing_remote_ip}/{peers_needing_remote_ip} IPs updated, {(max_loop * sleep_loop) - (loop_count * sleep_loop)}s remaining.")

    if loop_count >= max_loop:
        print_red(f"Warning: Could not get Remote IPs in {max_loop * sleep_loop} seconds.")
        return False
    else:
      return True


def run_integration_changes(change_list, fabricmaps, authenticated_sase_sdk, meraki_sdk):
    # start
    global USED_ENUMS
    global status_bar
    current_enum = 1
    
    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Iterating Commited Changes")

    # do changes first
    for change in change_list:
        change_is_delete = False
        if not change.get("network"):
            # change not valid, skip.
            print("STATUS: SKIPPING DUE TO ERROR")
            continue
        if not change.get("region"):
            # no region = delete
            # TODO do this
            change_is_delete = True
        # change.get("spn")

        # Extract fabricmap for this change (first, in case of bug where multiple match, but there should ALWAYS be a matching entry)
        fabricmap = [entry for entry in fabricmaps if entry.get("sdwan_name") == change.get("network")][0]
        # jd(fabricmap)

        if not change_is_delete:

            # pa ikegw first
            ikegw_change = deepcopy(PA_IKE_GATEWAY_TEMPLATE)
            ikegw_change["name"] = PA_PREAMBLE + PA_SEPERATOR + space_to_underscore(change.get("network"))
            ikegw_change["id"] = fabricmap.get("pa_ike_gw_id")
            ikegw_change["authentication"]["pre_shared_key"]["key"] = PSK
            # get ready for email.
            fm_enum = fabricmap.get("enum")
            if not fm_enum:
                # check if current enum is in used_enums
                while current_enum in USED_ENUMS:
                    # if so, keep incrementing until we get an unused enum.
                    current_enum += 1

                # set new enum from the pool.
                fm_enum = current_enum
                fabricmap['enum'] = current_enum
                USED_ENUMS.add(current_enum)
                # get ready for next enum pick
                current_enum += 1
            # add unique enum to sdwan side email
            ikegw_change["peer_id"]["id"] = enum_in_email(EMAIL_TEMPLATE, fm_enum)
            # PA side is always the same
            ikegw_change["local_id"]["id"] = "pa-headend-" + EMAIL_TEMPLATE
            ikegw_change["protocol"]["ikev2"]["ike_crypto_profile"] = PA_IKE_CRYPTO_PROFILE

            #jd(ikegw_change)
            # check if update or new.
            if ikegw_change['id']:
                # update
                put_id = ikegw_change['id']
                ikegw_change.pop('id', None)
                ikegw_result = put_pa_ike_gateways(authenticated_sase_sdk, put_id, ikegw_change)
                # jd_detailed(ikegw_result)
            else:
                # new, remove ID
                ikegw_change.pop('id', None)
                ikegw_result = post_pa_ike_gateways(authenticated_sase_sdk, ikegw_change)
            

            # pa ipsec tunnel next
            ipsec_tunnel_change = deepcopy(PA_IPSEC_TUNNEL_TEMPLATE)
            ipsec_tunnel_change["name"] = PA_PREAMBLE + PA_SEPERATOR + space_to_underscore(change.get("network"))
            ipsec_tunnel_change["id"] = fabricmap.get("pa_ipsec_tunnel_id")
            ipsec_tunnel_change["auto_key"]["ike_gateway"][0]["name"] = ikegw_change["name"]
            ipsec_tunnel_change["auto_key"]["ipsec_crypto_profile"] = PA_IPSEC_CRYPTO_PROFILE

            #jd(ipsec_tunnel_change)
            # check if update or new.
            if ipsec_tunnel_change['id']:
                # update
                put_id = ipsec_tunnel_change['id']
                ipsec_tunnel_change.pop('id', None)
                ipsec_tunnel_result = put_pa_ipsec_tunnels(authenticated_sase_sdk, put_id, ipsec_tunnel_change)
                # jd_detailed(ipsec_tunnel_result)
            else:
                # new, remove ID
                ipsec_tunnel_change.pop('id', None)
                ipsec_tunnel_result = post_pa_ipsec_tunnels(authenticated_sase_sdk, ipsec_tunnel_change)


            # pa rn next
            fabricmap['pa_rn_name'] = PA_PREAMBLE + PA_SEPERATOR + change.get("network")
            rn_change = deepcopy(PA_RN_TEMPLATE)
            rn_change["name"] = fabricmap['pa_rn_name']
            rn_change["id"] = fabricmap.get("pa_rn_id")
            rn_change["region"] = change.get('region')
            rn_change["spn_name"] = change.get('spn')
            rn_change["ipsec_tunnel"] = ipsec_tunnel_change["name"]
            rn_change["subnets"] = fabricmap.get('sdwan_subnets', [])

            #jd(ipsec_tunnel_change)
            # check if update or new.
            if rn_change['id']:
                # update
                put_id = rn_change['id']
                rn_change.pop('id', None)
                rn_result = put_pa_remote_networks(authenticated_sase_sdk, put_id, rn_change)
                # jd_detailed(rn_result)
            else:
                # new, remove ID
                rn_change.pop('id', None)
                rn_result = post_pa_remote_networks(authenticated_sase_sdk, rn_change)

            # at this point region/spn set - update fabricmap.
            fabricmap["pa_spn"] = change.get('spn')
            fabricmap["pa_region"] = change.get('region')

            # Ok, time to update Meraki Site
            meraki_site_tag = MERAKI_PREAMBLE + MERAKI_SEPERATOR + space_to_underscore(fabricmap["sdwan_name"])
            # remove existing PREAMBLE site tags, if they exist.
            candidate_meraki_site_tags = [tag for tag in fabricmap.get("sdwan_tags", []) if not tag.startswith(MERAKI_PREAMBLE + MERAKI_SEPERATOR)]
            candidate_meraki_site_tags.append(meraki_site_tag)
            meraki_site_result = dashboard.networks.updateNetwork(fabricmap["sdwan_id"], tags=candidate_meraki_site_tags)

            # jd(rn_result)
            


        # now handle deletes
        else:
            # double santiy check for delete
            if change.get("network") and change.get('region') is None and change.get('spn') is None:

                # update fabricmap to None so meraki VPNs won't get generated.
                fabricmap['pa_region'] = None
                fabricmap['pa_spn'] = None
                # Start with RNs
                delete_id = fabricmap.get("pa_rn_id")
                if delete_id:
                    rn_result = delete_pa_remote_networks(authenticated_sase_sdk, delete_id)
                    # jd_detailed(rn_result)

                # next ipsec tunnel
                delete_id = fabricmap.get("pa_ipsec_tunnel_id")
                if delete_id:
                    ipsec_tunnel_result = delete_pa_ipsec_tunnels(authenticated_sase_sdk, delete_id)
                    # jd_detailed(ipsec_tunnel_result)

                # finally ipsec gw
                delete_id = fabricmap.get("pa_ike_gw_id")
                if delete_id:
                    ikegw_result = delete_pa_ike_gateways(authenticated_sase_sdk, delete_id)
                    # jd_detailed(ikegw_result)
                
                # Ok, time to update Meraki Site
                # remove existing PREAMBLE site tags, if they exist.
                candidate_meraki_site_tags = [tag for tag in fabricmap.get("sdwan_tags", []) if not tag.startswith(MERAKI_PREAMBLE + MERAKI_SEPERATOR)]
                meraki_site_result = dashboard.networks.updateNetwork(fabricmap["sdwan_id"], tags=candidate_meraki_site_tags)

        # update status bar



    # Ok push the RN config so we can get some tunnel IPs.
    push_result = push_candidate_config(authenticated_sase_sdk)

    # update status
    refresh_globals()
    update_status()

    # jd(push_result)
    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Pushing Prisma Access Changes")
    # some time for the push to complete:
    time.sleep(10)

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Updating Meraki VPN IPs")

    # ok, now we need to do the silly meraki peers. I mean, who makes an API that:
    #  - Needs you to reconfigure all objects to add/delete/modify one.
    #  - Has a PSK that you can't get/set for each one.
    # (I firmly believe there's a better way I just don't know it - or this is the worst API ever..)
    candidate_meraki_peers, meraki_peer_name2rn_name = generate_meraki_peers_from_fabricmaps(fabricmaps)
            
    meraki_peer_result = dashboard.appliance.updateOrganizationApplianceVpnThirdPartyVPNPeers(meraki_org_id, candidate_meraki_peers)
    #jd(meraki_peer_result)
    
    # jd(candidate_meraki_peers)
    # Start loop to fill Meraki tunnel IPs.
    update_meraki_endpoint_ips(authenticated_sase_sdk, candidate_meraki_peers, meraki_peer_name2rn_name)

    # Final update status
    refresh_globals()
    update_status()

    # max_loop = 60
    # initial_peers_needing_remote_ip = len([peer for peer in candidate_meraki_peers if peer.get("publicIp") == "127.0.0.1"])
    # # print(initial_peers_needing_remote_ip)
    # previous_peers_needing_remote_ip = initial_peers_needing_remote_ip
    # peers_needing_remote_ip = initial_peers_needing_remote_ip
    # print(f"{peers_needing_remote_ip} of {initial_peers_needing_remote_ip} peers need remote IP update.")
    # while peers_needing_remote_ip > 0 and max_loop >= 0:
    #     # update dicts
    #     print(f"loop: {max_loop}")
    #     rn_ip_success, rn_name2ip, ip2rn_name, message = remote_gw_addr_dicts(authenticated_sase_sdk)
    #     for peer in candidate_meraki_peers:
    #         peer_name = peer.get("name", "")
    #         peer_ip = peer.get('publicIp')
    #         print(f"{peer_name} : {peer.get('publicIp')}")
    #         print(f'{peer_name} : {meraki_peer_name2rn_name.get(peer_name, "")}')
    #         print(f'{peer_name} : {rn_name2ip.get(meraki_peer_name2rn_name.get(peer_name, ""),"127.0.0.1")}')
    #         if peer_ip == "127.0.0.1":
    #             # try to update.
    #             peer['publicIp'] = rn_name2ip.get(meraki_peer_name2rn_name.get(peer_name, ""),"127.0.0.1")
    #     time.sleep(2)

    #     peers_needing_remote_ip = len([peer for peer in candidate_meraki_peers if peer.get("publicIp") == "127.0.0.1"])
    #     if peers_needing_remote_ip < previous_peers_needing_remote_ip:
    #         # update tracker and run API as we got at least one.
    #         previous_peers_needing_remote_ip = peers_needing_remote_ip
    #         meraki_peer_result = dashboard.appliance.updateOrganizationApplianceVpnThirdPartyVPNPeers(meraki_org_id, candidate_meraki_peers)
    #         #jd(meraki_peer_result)
    #         print(f"{peers_needing_remote_ip} of {initial_peers_needing_remote_ip} peers need remote IP update.")
    #     max_loop -= 1
    
    # if max_loop <= 0:
    #     print_red("Error: Could not get Remote IPs in 60 seconds.")
    return




# Templates

MERAKI_PEER_TEMPLATE = {
    "name": None,
    "publicIp": None,
    "privateSubnets": [ "0.0.0.0/0" ],
    "localId": None,
    "remoteId": None,
    "ikeVersion": "2",
    "networkTags": [],
    "ipsecPolicies": {
        "ikeCipherAlgo": [
            "aes256"
        ],
        "ikeAuthAlgo": [
            "sha1"
        ],
        "ikePrfAlgo": [
            "default"
        ],
        "ikeDiffieHellmanGroup": [
            "group2"
        ],
        "ikeLifetime": 28800,
        "childCipherAlgo": [
            "aes256",
            "aes192",
            "aes128"
        ],
        "childAuthAlgo": [
            "sha256",
            "sha1",
            "md5"
        ],
        "childPfsGroup": [
            "group5"
        ],
        "childLifetime": 3600
    }
}

PA_RN_TEMPLATE = {
    "id": None,
    "name": None,
    "folder": "Remote Networks",
    "license_type": "FWAAS-AGGREGATE",
    "region": None,
    "spn_name": None,
    "ipsec_tunnel": None,
    "subnets": [],
    "ecmp_load_balancing": "disable"
}

PA_IPSEC_TUNNEL_TEMPLATE = {
    "id": None,
    "name": None,
    "folder": "Remote Networks",
    "tunnel_interface": "tunnel",
    "auto_key": {
        "ike_gateway": [
            {
                "name": None
            }
        ],
        "ipsec_crypto_profile": None
    }
}

PA_IKE_GATEWAY_TEMPLATE = {
    "id": None,
    "name": None,
    #"folder": "Remote Networks",
    "local_address": {
        "interface": "vlan"
    },
    "peer_address": {
        "dynamic": {}
    },
    "authentication": {
        "pre_shared_key": {
            "key": None
        }
    },
    "peer_id": {
        "type": "ufqdn",
        "id": None
    },
    "local_id": {
        "type": "ufqdn",
        "id": "prisma-tunnel@mycompany.com"
    },
    "protocol": {
        "version": "ikev2",
        "ikev1": {
            "dpd": {
                "enable": True
            }
        },
        "ikev2": {
            "dpd": {
                "enable": True
            },
            "ike_crypto_profile": None
        }
    },
    "protocol_common": {
        "nat_traversal": {
            "enable": True
        },
        "fragmentation": {
            "enable": False
        },
        "passive_mode": True
    }
}


def refresh_globals():

    global USED_ENUMS
    global meraki_orgs
    global meraki_org_id
    global meraki_networks
    global meraki_network_names
    global meraki_thirdpartyvpnpeers
    global meraki_peers
    global pa_allocatable_regions
    global display2region
    global pa_remote_networks
    global pa_ike_crypto_profiles
    global pa_ipsec_crypto_profiles
    global pa_ike_gateways
    global pa_ipsec_tunnels
    global fabricmaps
    global pa_available_regions
    global status_bar


    USED_ENUMS = set()

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Loading Meraki Info")

    # Meraki globals
    meraki_orgs = dashboard.organizations.getOrganizations()
    meraki_org_id = meraki_orgs[0].get('id')
    meraki_networks = dashboard.organizations.getOrganizationNetworks(meraki_org_id)
    # sort by name
    meraki_networks.sort(key=lambda k : k.get('name', ''))
    meraki_network_names = [ x.get('name') for x in meraki_networks]
    meraki_thirdpartyvpnpeers = dashboard.appliance.getOrganizationApplianceVpnThirdPartyVPNPeers(meraki_org_id)
    meraki_peers = meraki_thirdpartyvpnpeers.get("peers") 
    #meraki_network_names.sort()


    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Loading Prisma Access Info")


    # PA globals
    pa_allocatable_regions, display2region = get_pa_allocatable_regions(sdk)
    pa_remote_networks = get_pa_remote_networks(sdk)
    pa_ike_crypto_profiles = get_pa_ike_crypto_profiles(sdk)
    pa_ipsec_crypto_profiles = get_pa_ipsec_crypto_profiles(sdk)
    pa_ike_gateways = get_pa_ike_gateways(sdk)
    pa_ipsec_tunnels = get_pa_ipsec_tunnels(sdk)

    fabricmaps = refresh_fabricmap(meraki_networks, meraki_peers, pa_allocatable_regions,
                                  pa_remote_networks, pa_ike_crypto_profiles, pa_ipsec_crypto_profiles,
                                  pa_ike_gateways, pa_ipsec_tunnels, dashboard)
    #jd(fabricmaps)


    pa_available_regions = [x for x in pa_allocatable_regions.keys()]
    pa_available_regions.sort()
    pa_available_regions.insert(0, DISABLE_STRING)

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Finished Loading Prisma Access Info")

    return

# load the globals above
refresh_globals()

###                         ###
### START WIDGET / UI stuff ###
###                         ###

# static widget stuff:

button_layout = widgets.Layout(width='auto', height='40px')
button_status_layout = widgets.Layout(width='auto', height='40px')
status_layout = widgets.Layout(width='auto', height='30px')
label_pulldown_layout = widgets.Layout(width='auto', height='30px')

labels_header = widgets.Button(description='Meraki Network', 
                                disabled=True,
                                layout=button_layout)
regions_header = widgets.Button(description='Prisma Access Region', 
                                disabled=True,
                                layout=button_layout)
spns_header = widgets.Button(description='Prisma Access SPN', 
                                disabled=True,
                                layout=button_layout)
force_commit_header = widgets.Button(description='Force Commit', 
                                disabled=True,
                                layout=button_layout)
pa_config_status_header = widgets.Button(description='Prisma Status', 
                                disabled=True,
                                layout=button_status_layout)
meraki_config_status_header = widgets.Button(description='Meraki Status', 
                                disabled=True,
                                layout=button_status_layout)
tunnel_status_header = widgets.Button(description='Tunnel Status', 
                                disabled=True,
                                layout=button_status_layout)

for tmp_widget in [labels_header, regions_header, spns_header, force_commit_header, meraki_config_status_header,
                   pa_config_status_header, tunnel_status_header]:
    tmp_widget.style.font_weight = 'bold'

labels_header_list=[labels_header]
regions_header_list=[regions_header]
spns_header_list=[spns_header]
force_commit_header_list = [force_commit_header]
pa_config_status_header_list = [pa_config_status_header]
meraki_config_status_header_list = [meraki_config_status_header]
tunnel_status_header_list = [tunnel_status_header]

def set_warning(button):
    button.style={'button_color':'#ff9900'}

def set_good(button):
    button.style={'button_color':'#009933'}

def set_bad(button):
    button.style={'button_color':'#cc0000'}

def set_unknown(button):
    button.style={'button_color':'#f7f7f7'}


def disable_all_widgets(event):
    # Disable table of networks
    for idx, widget_entry in enumerate(wgt_labels):
          widget_entry.disabled = True
          wgt_regions[idx].disabled = True
          wgt_spns[idx].disabled = True
          wgt_force_commit[idx].disabled = True
    # disable buttons
    commit_box_widgets[0].disabled = True
    for button_item in action_box_widgets:
        button_item.disabled = True
    return

def reenable_all_widgets(event):
    # Re-enable table of networks
    for idx, widget_entry in enumerate(wgt_labels):
          widget_entry.disabled = False
          wgt_regions[idx].disabled = False
          wgt_spns[idx].disabled = True if wgt_regions[idx].value == DISABLE_STRING else False
          wgt_force_commit[idx].disabled = False
    # re-enable buttons
    for wdgt in action_box_widgets:
        wdgt.disabled = False
    commit_box_widgets[0].disabled = False
    for button_item in action_box_widgets:
        button_item.disabled = False 
    # after re-enable, check and make sure commit widget rules should be run
    show_hide_commit_bar()
    return  


def detect_conf_change(index):
    # Checks if the values in the row for the Meraki site / pa selections are different than what was in the original config
    config_dict = fabricmaps[index]  # fabricmap index was created from meraki_network_names.
    config_dict_region = config_dict.get('pa_region') if config_dict.get('pa_region') else DISABLE_STRING
    config_dict_spn = config_dict.get('pa_spn') if (config_dict.get('pa_spn') and config_dict.get('pa_region')) else DISABLE_STRING
    #config_dict=original_config.get(meraki_network_names[index])
    region_selected = wgt_regions[index].value
    spn_selected = wgt_spns[index].value
    force_commit_selected = wgt_force_commit[index].value
    # print(f"{region_selected} == {config_dict_region}")
    # print(f"{spn_selected} == {config_dict_spn}")
    # set site label back to non-modified original name - add modified only if change detected.
    wgt_labels[index].value = config_dict.get('sdwan_name')
    if config_dict_region != region_selected or config_dict_spn != spn_selected or force_commit_selected == True:
      wgt_labels[index].value += " (Modified)"
      return True

def show_hide_commit_bar():
    # show / hide commit bar based on presense of any (modified) text in label.
    # EDIT - now disable commit/revert instead of hide.
    show_bar = False
    for idx, network in enumerate(meraki_network_names):
      # check for any (modified) tags
      if wgt_labels[idx].value != network:
        show_bar = True
    if show_bar:
      commit_box_widgets[0].disabled = False
      action_box_widgets[0].disabled = False
      #commit_box.layout.visibility = 'visible'
    else:
      # clear output in output widget on hide/disable
      with output_widget:
          clear_output()
      commit_box_widgets[0].disabled = True
      action_box_widgets[0].disabled = True
      #commit_box.layout.visibility = 'hidden'

def force_commit_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Get index of change
        index = wgt_force_commit.index(change.owner)
        changed = detect_conf_change(index)

def region_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Get index of change
        index = wgt_regions.index(change.owner)
        # update SPN pulldown
        if change['new'] == DISABLE_STRING:
          wgt_spns[index].disabled = True
          wgt_spns[index].options = [DISABLE_STRING]
        else:
          wgt_spns[index].disabled = False
          wgt_spns[index].options = [DISABLE_STRING] + pa_allocatable_regions.get(change['new'],{}).get("spn_name_list",[])
        # Mark modified if so
        changed = detect_conf_change(index)
        
def spn_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        index = wgt_spns.index(change.owner)
        # Mark modified if so
        changed = detect_conf_change(index)

def label_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # if one is modified, show/hide commit bar
        show_hide_commit_bar()


def on_reset_changes_clicked(event):
    global status_bar
    status_bar = 0
    # reset the widgets.
    for idx, fabric in enumerate(fabricmaps):
          wgt_labels[idx].value=fabric["sdwan_name"]
          wgt_regions[idx].value=fabric.get("pa_region") if fabric.get("pa_region") is not None else DISABLE_STRING
          wgt_spns[idx].value=fabric.get("pa_spn") if fabric.get("pa_spn") is not None else DISABLE_STRING
          wgt_spns[idx].disable = True if fabric.get("pa_spn") is None else False
          wgt_force_commit[idx].value = False
    return

def on_refresh_status(event):
    global status_bar
    status_bar = tqdm(total=2, leave=False)
    status_bar.set_description("Refreshing Status")
    with output_widget:
        clear_output()
        update_status()
        status_bar.close()
    return

def on_refresh_all(event):
    global status_bar
    status_bar = tqdm(total=10, leave=False)
    status_bar.set_description("Refreshing Interface")
    disable_all_widgets(event)
    refresh_globals()
    main_box.close()
    action_box.close() 
    commit_box.close()
    output_box.close()
    main_loop()
    status_bar.close()
    return

def on_refresh_vpn_ip_clicked(event):
    global status_bar
    status_bar = tqdm(total=9, leave=False)
    status_bar.set_description("Refreshing VPN IPs")
    with output_widget:
        clear_output()
        disable_all_widgets(None)
        # my_list = list(range(10))
        # with tqdm(total=len(my_list), leave=False) as pbar:
        #   for x in my_list:
        #       time.sleep(1) 
        #       pbar.set_description(f"Processing {x}")
        #       pbar.update(1)
        refresh_globals()
        candidate_meraki_peers, meraki_peer_name2rn_name = generate_meraki_peers_from_fabricmaps(fabricmaps)
        update_meraki_endpoint_ips(sdk, candidate_meraki_peers, meraki_peer_name2rn_name, force=True)
        reenable_all_widgets(None)
        status_bar.close()
    return

def on_commit_clicked(event):
    global status_bar
    status_bar = tqdm(total=16, leave=False)
    status_bar.set_description("Performing Commit")
    output_changes = []
    error_list =[]
    for idx, network_name in enumerate(meraki_network_names):
        # check if this line was modified. If a modified line, label should be different. If not modified, skip.
        # print(f"{wgt_labels[idx].value} == {network_name}")
        if wgt_labels[idx].value == network_name:
            # no changes. Skip.
            continue
        selected_region = wgt_regions[idx].value if wgt_regions[idx].value is not DISABLE_STRING else None
        selected_spn = wgt_spns[idx].value if wgt_spns[idx].value is not DISABLE_STRING else None

        if selected_region and not selected_spn:
            # SPN is required if region. throw error.
            error_list.append(f"Network '{network_name}' has region '{selected_region}' but no SPN. Please select an SPN.")

        output_changes.append({
            "network": network_name,
            "region": selected_region,
            "spn": selected_spn
        })
    # were there any changes?
    if not output_changes:
        error_list.append(f"No changes to commit. Please make changes and try again.")

    with output_widget:
        clear_output() # clear last info
        # quick config check
        if error_list: # were errors seen?
            print_red("Error:")
            for error in error_list:
                print_red(error)
            return # early exit.
            
        # jd(output_changes)
        disable_all_widgets(None)
        # my_list = list(range(10))
        # with tqdm(total=len(my_list), leave=False) as pbar:
        #   for x in my_list:
        #       time.sleep(1) 
        #       pbar.set_description(f"Processing {x}")
        #       pbar.update(1)
        result = run_integration_changes(output_changes, fabricmaps, sdk, dashboard)
        reenable_all_widgets(None)
    
    status_bar.close()
    return


# Get current config - replaced by fabricmap


# for network in meraki_network_names:
#   current_config[network] = {
#       "network": network,
#       "region": DISABLE_STRING,
#       "spn": DISABLE_STRING
#   }
  # current_config[network] = {
  #     "network": network,
  #     "region": "us-northwest",
  #     "spn": "us-northwest-yew"
  # }

# make backup of current state for change check.
#original_config = deepcopy(current_config)

def update_status(event=None):
    global status_bar

    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Updating Network Status")

    for idx, fabricmap in enumerate(fabricmaps):
        set_unknown(wgt_meraki_status[idx])
        set_unknown(wgt_pa_status[idx])
        set_unknown(wgt_tunnel_status[idx])
    time.sleep(0.25)
    for idx, fabricmap in enumerate(fabricmaps):
        meraki_status = fabricmap.get("sdwan_config_status")
        pa_status = fabricmap.get("pa_config_status")
        tunnel_status = fabricmap.get("tunnel_status")
        # with output_widget:
        #     print(f"{fabricmap.get('sdwan_name')} STATUS: Meraki - {meraki_status}, pa - {pa_status}, tunnel - {tunnel_status}")
        if meraki_status == "good":
            set_good(wgt_meraki_status[idx])
        elif meraki_status == "bad":
            set_bad(wgt_meraki_status[idx])
        elif meraki_status == "warn":
            set_warning(wgt_meraki_status[idx])
        else:
            set_unknown(wgt_meraki_status[idx])

        if pa_status == "good":
            set_good(wgt_pa_status[idx])
        elif pa_status == "bad":
            set_bad(wgt_pa_status[idx])
        elif pa_status == "warn":
            set_warning(wgt_pa_status[idx])
        else:
            set_unknown(wgt_pa_status[idx])

        
        if tunnel_status == "good":
            set_good(wgt_tunnel_status[idx])
        elif tunnel_status == "bad":
            set_bad(wgt_tunnel_status[idx])
        elif tunnel_status == "warn":
            set_warning(wgt_tunnel_status[idx])
        else:
            set_unknown(wgt_tunnel_status[idx])
    if status_bar is not None:
        status_bar.update(1)
        status_bar.set_description("Finished Updating Network Status")



def main_loop():
    global wgt_labels
    global wgt_regions
    global wgt_spns
    global wgt_force_commit
    global wgt_meraki_status
    global wgt_pa_status
    global wgt_tunnel_status
    global current_config
    global output_widget
    global commit_box_widgets
    global action_box_widgets
    global main_box
    global action_box
    global commit_box
    global output_box

    wgt_labels = []
    wgt_regions = []
    wgt_spns = []
    wgt_force_commit = []
    wgt_meraki_status = []
    wgt_pa_status = []
    wgt_tunnel_status = []
    current_config = {}
    for fabric in fabricmaps:
        # fabricmaps shoud be in same order as meraki_network_names, so meraki_network_names should = [entry["sdwan_name"] for entry in fabricmaps]
        wgt_labels.append(widgets.Label(value=fabric["sdwan_name"], layout=label_pulldown_layout))
        wgt_regions.append(widgets.Dropdown(
                          name=fabric["sdwan_name"],
                          options=pa_available_regions,
                          value=fabric.get("pa_region") if fabric.get("pa_region") is not None else DISABLE_STRING,
                          description="",
                          disabled=False,
                          layout=label_pulldown_layout
                      ))
        wgt_spns.append(widgets.Dropdown(
                      name=fabric["sdwan_name"],
                      options=[DISABLE_STRING] + pa_allocatable_regions.get(fabric.get("pa_region",{}),{}).get("spn_name_list",[]),
                      value=fabric.get("pa_spn") if (fabric.get("pa_spn") is not None) and (fabric.get("pa_region") is not None) else DISABLE_STRING,
                      description="",
                      disabled=True if fabric.get("pa_spn") is None or fabric.get("pa_region") is None else False,
                      layout=label_pulldown_layout
                  ))
        wgt_force_commit.append(widgets.Checkbox(value=False, indent=False, layout=label_pulldown_layout))
        wgt_pa_status.append(widgets.Button(name=fabric["sdwan_name"], layout=status_layout))
        wgt_meraki_status.append(widgets.Button(name=fabric["sdwan_name"], layout=status_layout))
        wgt_tunnel_status.append(widgets.Button(name=fabric["sdwan_name"], layout=status_layout))

    region_box = widgets.VBox(regions_header_list + wgt_regions)
    force_commit_box = widgets.VBox(force_commit_header_list + wgt_force_commit, layout=widgets.Layout( display='flex' , align_items='center'))
    pa_status_box = widgets.VBox(pa_config_status_header_list + wgt_pa_status)
    meraki_status_box = widgets.VBox(meraki_config_status_header_list + wgt_meraki_status)
    tunnel_status_box = widgets.VBox(tunnel_status_header_list + wgt_tunnel_status)

    action_box_widgets = [widgets.Button(description='Revert Pending Changes', disabled=True, layout=button_layout),
                          widgets.Button(description='Hard Refresh All', disabled=False, layout=button_layout),
                          widgets.Button(description='Force VPN-IP Update', disabled=False, layout=button_layout),
                          widgets.Button(description='Refresh Status', disabled=False, layout=button_layout)]
    commit_box_widgets = [widgets.Button(description='Commit', disabled=True, layout=button_layout),
                          widgets.Output()]
    output_box_widgets = [widgets.Output()]
    action_box = widgets.HBox(action_box_widgets)
    commit_box = widgets.HBox(commit_box_widgets)
    output_box = widgets.HBox(output_box_widgets)
    #commit_box.layout.visibility = 'hidden'

    # add events to widgets
    for i in wgt_regions:
      i.observe(region_on_change)

    for i in wgt_spns:
      i.observe(spn_on_change)

    for i in wgt_labels:
      i.observe(label_on_change)

    for i in wgt_force_commit:
      i.observe(force_commit_on_change)

    output_widget = output_box_widgets[0]

    commit_box_widgets[0].on_click(on_commit_clicked)
    action_box_widgets[0].on_click(on_reset_changes_clicked)
    action_box_widgets[1].on_click(on_refresh_all)
    action_box_widgets[2].on_click(on_refresh_vpn_ip_clicked)
    action_box_widgets[3].on_click(on_refresh_status)

    spn_box = widgets.VBox(spns_header_list + wgt_spns)

    # update status
    update_status()
    main_box = widgets.HBox([widgets.VBox(labels_header_list + wgt_labels), region_box, spn_box, force_commit_box, pa_status_box, meraki_status_box, tunnel_status_box])
    display(main_box, action_box, commit_box, output_box)

main_loop()
status_bar.close()

# = tqdm(total=len(tqdm_tasks), leave=False)
# my_list = list(range(10))
# with tqdm(total=len(my_list), leave=False) as pbar:
#   for x in my_list:
#       time.sleep(1) 
#       pbar.set_description(f"Processing {x}")
#       pbar.update(1)

###                       ###
### END WIDGET / UI stuff ###
###                       ###

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />

# **DEBUGS / TEST CELLS BELOW HERE. NOTHING GREAT.**



In [ ]:
print("hello world")